In [1]:
import pandas
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 
import mlflow
from mlflow.models import infer_signature

In [2]:
mlflow.set_tracking_uri(uri='http://127.0.0.1:5000')

In [3]:
X,y=datasets.load_iris(return_X_y=True)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

### using perameter 1

In [4]:
parameter={"penalty":'l2', "solver": "lbfgs", "max_iter": 1000, "multi_class": "auto", "random_state": 8888}

In [5]:
model=LogisticRegression(**parameter)
model.fit(X_train,y_train)
model

/data/temp/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888)

In [6]:
y_pred=model.predict(X_test)
y_pred

array([1, 0, 2, 1, 1, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 0, 2, 0, 2, 0, 2, 0,
       1, 0, 1, 1, 2, 1, 1, 1])

In [7]:
accuracy=accuracy_score(y_test,y_pred)
accuracy

1.0

In [8]:
mlflow.set_experiment("mlflow_1")

2025/02/04 16:44:41 INFO mlflow.tracking.fluent: Experiment with name 'mlflow_1' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/403252831614015449', creation_time=1738667681285, experiment_id='403252831614015449', last_update_time=1738667681285, lifecycle_stage='active', name='mlflow_1', tags={}>

In [9]:
with mlflow.start_run():
    mlflow.log_params(parameter)
    mlflow.log_metric("accuracy",accuracy)
    mlflow.set_tag("Training info","basic model on iris dataset")

    signature=infer_signature(X_train,model.predict(X_train))
    model_info=mlflow.sklearn.log_model(
        sk_model= model,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-1"
    ) 

2025/02/04 16:44:41 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

Successfully registered model 'tracking-1'.
2025/02/04 16:44:47 INFO mlflow.store.model_registry.abstrac

🏃 View run charming-ox-542 at: http://127.0.0.1:5000/#/experiments/403252831614015449/runs/4273d4f3677641048b8c42144076011d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/403252831614015449


Created version '1' of model 'tracking-1'.


### using parameter 2

In [10]:
parameter={ "solver": "newton-cg", "max_iter": 1000, "multi_class": "auto", "random_state": 8888}

In [11]:
model=LogisticRegression(**parameter)
model.fit(X_train,y_train)
model

/data/temp/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888,
                   solver='newton-cg')

In [12]:
y_pred=model.predict(X_test)
y_pred

array([1, 0, 2, 1, 1, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 0, 2, 0, 2, 0, 2, 0,
       1, 0, 1, 1, 2, 1, 1, 1])

In [13]:
accuracy=accuracy_score(y_test,y_pred)
accuracy

1.0

In [17]:
with mlflow.start_run():
    mlflow.log_params(parameter)
    mlflow.log_metric("accuracy",accuracy)
    mlflow.set_tag("Training info","basic model on iris dataset")

    signature=infer_signature(X_train,model.predict(X_train))
    model_info=mlflow.sklearn.log_model(
        sk_model= model,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-1"
    )  

Successfully registered model 'tracking-2'.
2025/02/04 16:47:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-2, version 1


🏃 View run crawling-hound-572 at: http://127.0.0.1:5000/#/experiments/403252831614015449/runs/f0258f02bae043cfaf3be89d2430a5b0
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/403252831614015449


Created version '1' of model 'tracking-2'.


In [15]:
model_info.model_uri

'runs:/593e955de0d8446584c5530617bad7ec/iris_model'

### load model back for prediction as generic python function model

In [16]:
loaded_model=mlflow.pyfunc.load_model(model_info.model_uri)
prediction=loaded_model.predict(X_test)

iris_feature_name=datasets.load_iris().feature_names   
result=pandas.DataFrame(X_test,columns=iris_feature_name)
result["prediction"]=prediction
result["actual"]=y_test

result

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),prediction,actual
0,5.5,2.3,4.0,1.3,1,1
1,4.7,3.2,1.6,0.2,0,0
2,6.7,3.3,5.7,2.5,2,2
3,5.2,2.7,3.9,1.4,1,1
4,5.8,2.7,4.1,1.0,1,1
5,5.1,3.8,1.5,0.3,0,0
6,5.8,2.6,4.0,1.2,1,1
7,5.9,3.0,5.1,1.8,2,2
8,6.7,3.1,5.6,2.4,2,2
9,6.5,3.0,5.5,1.8,2,2


In [20]:
import mlflow.sklearn 

model_name="tracking-1"
model_version="letest"
model_uri=f'model:/{model_name}/{model_version}'
model_uri
# model=mlflow.sklearn.load_model(model_uri)
# model

'model:/tracking-1/letest'